# Lab : Doing XYZ
Overview of lab

In [ ]:
## basic imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [ ]:
## Google Colab check
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

print ("Running in Google COLAB : ", IN_COLAB)


if IN_COLAB == True:
    from tensorboardcolab import *
    !pip install -U tensorboardcolab
# Load the TensorBoard notebook extension
    %load_ext tensorboard

## Step 1: Get Data
Describe the dataset

In [ ]:
## download the data
data_location = '/data/iris/keras/iris.csv'
# data_location = "https://s3.amazonaws.com/elephantscale-public/data/iris/keras/iris.csv"

data = pd.read_csv(data_location)

data

## Step 2 : Basic Exploration of Data

In [ ]:
## do datacleanup here
data = data.dropna()

## may be do some graphs ..etc

## Step 3 : Shape the Data

In [ ]:
input_columns = ['a', 'b']
x = data [input_columns]
y = data[['label_col']]

print (x.head())
print (y.head())

## encode data if need to

### Create train / test split

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0) 

print ("x_train.shape : ", x_train.shape)
print ("y_train.shape : ", y_train.shape)
print ("x_test.shape : ", x_test.shape)
print ("y_test.shape : ", y_test.shape)

## Step 4 : Build the model

In [ ]:
model = tf.keras.Sequential([
            tf.keras.layers.Dense(units=5, activation=tf.nn.relu, input_dim=input_dim, name="input_layer"),
            tf.keras.layers.Dense(units=64, activation=tf.nn.relu, name="hidden_1"),
            tf.keras.layers.Dense(units=3,  activation=tf.nn.softmax, name="output_layer")
            ])

# loss = 'sparse_categorical_crossentropy'  or 'categorical_crossentropy'
model.compile(loss='sparse_categorical_crossentropy', 
                 optimizer=tf.keras.optimizers.Adam(), # or 'adam', 
                 metrics=['accuracy'])

print (model.summary())

## Step 5 : Setup Tensorboard

In [ ]:
## This is fairly boiler plate code

import datetime
import os

app_name = 'sample-1' # you can change this, if you like

tb_top_level_dir= '/tmp/tensorboard-logs'
tensorboard_logs_dir= os.path.join (tb_top_level_dir, app_name, 
                                    datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%S"))
print ("Saving TB logs to : " , tensorboard_logs_dir)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_logs_dir, histogram_freq=1)


## Step 6 : Train

In [ ]:
%%time

epochs = 100 

print ("training starting ...")
history = model.fit(
              x_train, y_train,
              epochs=epochs, validation_split = 0.2, verbose=1,
              callbacks=[tensorboard_callback])

print ("training done.")

## Step 7 : Plot train history

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

## Step 8 : Predict

In [ ]:
predictions = model.predict(x_test)
predictions

## Step 9 : Evaluate

In [ ]:
metrics = model.evaluate(x_test, y_test, verbose=0)

### 9.1 - Print out eval metrics

In [ ]:
metric_names = model.metrics_names
print ("model metrics : " , metric_names)

for idx, metric in enumerate(metric_names):
    print ("Metric : {} = {:,.2f}".format (metric_names[idx], metrics[idx]))

### 9.2 - Confusion matrix (for classification problems)

In [ ]:
## plain confusion matrix 

from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred, labels = [0,1,2])
cm

## Step 10 : Discuss results and Tune the Model
Class discussion on how the model performed and how to improve it.  
May be add more neurons / layers ..etc